# This notebook can be executed after getRegionalBox.ipynb

In [1]:
using Pkg
using Metal

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv"  # maybe GeoPoints and planet1D should be fusioned

# batchGPU should be at this level (I have not made it as a module yet, since the choice of Metal/CUDA should be done in a manual way)
include("../src/batchFiles/batchGPU.jl")


include("../src/commonBatchs.jl")
include("../src/flexOPT.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

using .commonBatchs, .flexOPT, .planet1D, .GeoPoints

  Activating 

devs = Metal.devices() = Metal.MTL.MTLDeviceInstance[Metal.MTL.MTLDeviceInstance (object of type AGXG13XDevice)]

project at `~/Documents/Github/flexOPT`



→ Using Metal backend (1 device(s))
Selected backend type: MetalBackend
  1.666555 seconds (3.83 M allocations: 173.427 MiB, 99.97% compilation time)


# give me a model of material variables (like seismic models)

In [2]:
# what you need as a model:

boxGrids = lazyProduceOrLoad("MarmousiCoordInfo"); # for a big model, I do not recommend to store these data

seismicModelMarmousi=lazyProduceOrLoad("seismicModelMarmousi");

This is the laziest ProduceOrLoad
Loading from ./tmp/MarmousiCoordInfo.jld2
This is the laziest ProduceOrLoad
Loading from ./tmp/seismicModelMarmousi.jld2


# choose your physics and get the semi symbolic OPT expression(s) 

In [3]:
famousEquationType="2DacousticTime"
@show myEquationInside=famousEquations(famousEquationType)


myEquationInside = famousEquations(famousEquationType) = (exprs = Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), fields = u(x, y, t), vars = v(x, y), extexprs = f(x, y, t), extfields = f(x, y, t), extvars = 1, coordinates = (x, y, t), ∂ = Any[Differential(x), Differential(y), Differential(t)], ∂² = Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])


(exprs = Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), fields = u(x, y, t), vars = v(x, y), extexprs = f(x, y, t), extfields = f(x, y, t), extvars = 1, coordinates = (x, y, t), ∂ = Any[Differential(x), Differential(y), Differential(t)], ∂² = Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])

In [4]:
Δnum = (1.0,1.0,1.0) # this should be in the same order as myEquationInside.coordinates 

# or
 
Δnum = (boxGrids.Δx,boxGrids.Δz,1.0) # hahaha this Δnum is not very good since there is no reason for the grids to be rectangular with this isotropic medium but let's try


(11.485642946317103, 4.444444444444445, 1.0)

# hereafter are the parameters which are important for modellers but maybe not interesting (so default numbers are given)

In [5]:
# concerning the order of test B-spline functions (-1 denotes a pulse function)
orderBtime=1
orderBspace=1
# the number of neighbouring points used to construct operators (except the point itself)
pointsInSpace=2
pointsInTime=2
# in order to control higher order partials, we need the supplemantary order > 0, for an ill-posed inversion of Taylor coefficients
supplementaryOrder=2
# B-spline order for the interpolated Taylor expansion
WorderBspace=1
WorderBtime=1

TaylorOptions=(WorderBtime=WorderBtime,WorderBspace=WorderBspace,supplementaryOrder=supplementaryOrder)
trialFunctionsCharacteristics=(orderBtime=orderBtime,orderBspace=orderBspace,pointsInSpace=pointsInSpace,pointsInTime=pointsInTime)



(orderBtime = 1, orderBspace = 1, pointsInSpace = 2, pointsInTime = 2)

In [6]:
OPTobj(myEquationInside, Δnum; TaylorOptions=TaylorOptions, trialFunctionsCharacteristics=trialFunctionsCharacteristics)

(bigα, varM) = (Any[Any[(node = -(v(x, y)^2), nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(3, 1, 1)), (node = -(v(x, y)^2), nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(1, 3, 1)), (node = 1, nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(1, 1, 3))];;], Any[v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉ v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉ v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉])


┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/taylorCoefInv/taylorCoefInv_13de2d0c.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:106


modifiedμ_vector = Float64.(car2vec(μpointsIndices[μ_oneD])) = [1.0, 1.0, 1.0]
(typeof(aa), aa, size(aa)) = (Matrix{Float64}, [1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0; 1.0 594054.84643979 8.940631796556378e6 1.0 594054.84643979 8.940631796556378e6 1.0 594054.84643979 8.940631796556378e6 1.0 594054.84643979 8.940631796556378e6 1.0 594054.84643979 8.940631796556378e6 1.0 594054.84643979 8.940631796556378e6 1.0 594054.84643979 8.940631796556378e6 1.0 594054.84643979 8.940631796556378e6 1.0 594054.84643979 8.940631796556378e6; 1.0 8.940631796556378e6 1.3875616367000288e8 1.0 8.940631796556378e6 1.3875616367000288e8 1.0 8.940631796556378e6 1.3875616367000288e8 1.0 8.940631796556378e6 1.3875616367000288e8 1.0 8.940631796556378e6 1.3875616367000288e8 1.0 8.940631796556378e6 1.3875616367000288e8 1.0 8.940631796556378e6 1.3875616367000288e8 1.0 8.940631796556378e6 1.3875616367000288e8 1.0 8.940631796556378e6 1.3875616367000288e

┌ Warning: The Git repository ('/Users/nobuaki/Documents/Github/flexOPT') is dirty! Appending -dirty to the commit ID.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_tools.jl:71
┌ Info: File /Users/nobuaki/Documents/Github/flexOPT/data/taylorCoefInv/taylorCoefInv_13de2d0c.jld2 saved.
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/2QF5p/src/saving_files.jl:115


UndefVarError: UndefVarError: `Adapt` not defined in `Main.flexOPT`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name also exists in Adapt.